In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras

# The Vanishing/Exploding Gradients Problem:
When an ANN has many layers, it becomes possible for the backpropagrated error signal to become very small, the point of vanishing, or very large, to the point of exceeding the memory limit of float datatypes, as the backward backprop sweep gets closer to the input layer. Both cases stop the network from being able to find a solution to the problem at hand. So, to solve this problem, we must configure the network architecture such that the gradients are not capable (or at least far less likely) to either vanish or blow up.

Considering that the loss function is typically computed by taking a measure analogous to the variance (e.g., mse), it seems reasonable that a solution to this problem would be to ensure that the variance of inputs to a layer should be roughly equal to the variance of the outputs of the same layer. We call the number of inputs to a layer the fan-in of the layer, and the number of outputs the fan-out. We control the variance of the inputs and outputs by randomly initializing the weight matrix subject to the following constraints; for each layer, where fan-avg = (fan-in + fan-out)/2, we have the weights distributed normally with mean 0 and variance 1/fan-avg. There are also other constraints we can specify that get the job done, but they are similar.

### Nonsaturating activation functions
Part of the prolificness of the vanishing/exploding gradients problem was the popular choice of activation function, namely the sigmoid activation function, which has saturating slopes as abs(y) approaches 1. This killed the gradients that were close to outputting 1 or -1, and so caused gradient vanishing. A solution to this problem was to more widely use nonsaturating activation functions, such as relu.

There exist many variations of relu, however. These variations were spawned because relu is not perfect. In fact, the greatest drawback of relu is that many neurons can effectively "die"--if their activation function reaches zero because many of their input weights happen to be negative, then the partial derivatives of that unit go to zero as well, and so the neuron ceases to change. Thus, we can knock out whole swaths of neurons from our networks using relu. Thus, the first solution that may come to mind would be to have some negative output on the relu function, which is called **leaky relu**, which is simply relu except for inputs x < 0, relu(x) = tx where t is some small number, 0.05 typically. Furthermore, there is **randomized leaky relu**, which chooses a random value of t. Next, we have **parametric leak relu**, which makes relu not a hyperparameter but rather a parameter of the network itself, and so the network learns the optimal value of t for fitting the training set. However, this can lead to overfitting, and so it's not usually a good choice on smaller training sets. Next, we have **elu**, i.e., the **exponential linear unit**. This is simply relu for x > 0, but is t * exp(x) - 1 for x <= 0. Lastly, we have the **scaled elu** i.e., **selu**. Selu is very similar to elu, except there is a scaling factor r >= 1 and x <= 0 is a little different:

                             selu(x) = if x > 0 then r * x; if x <= 0 then r * t * (e^x - 1).

Selu has the property of keeping the weights normalized, so that gradients do not explode or vanish. Thus, for deep feedforwad neural nets (FNNs), selu is often the best choice. However, selu cannot promise normalization of weights if the neural network is not feedforwad, thought often selu still performs well.

Lastly, the textbook states that generally selu > elu > leaky relu (and its variants Prelu and RRelu) > relu > tanh > sigmoid. Relu functions are generally faster to compute than elu functions, since they do not rely on the transcendental function e^x. In the following code, I show how to implement selu and leaky relu. It's important we choose the right weight initialization scheme, whether that be lecun initialization or he initialization.

In [2]:
# For LeakyReLU
model = keras.Sequential([...
                        keras.layers.Dense(n_neurons, kernel_initializer='he_normal'),
                        keras.layers.LeakyReLU(alpha = 0.01), ...])

SyntaxError: invalid syntax (<ipython-input-2-490d47edebab>, line 3)

In [3]:
# For SeLU, we add a layer like the following one to our sequential model (no skip layers and no recurrence)
SeluLayer = keras.layers.Dense(100, kernel_initializer='lecun_normal', activation='selu')

### Batch Normalization:
The story we've shared above increases the likelihood that we start training without vanishing or exploding gradients. However, small differences can be amplified over training, causing explosion or vanishment. Batch normalization is a way of trying to stop this and speeding up training. This works by normalizing the batch that we plug into the next layer and performing an affine transformation on the centered and normalized batch; that is, after centering and normalizing the mini batch that we are using as an input to the next layer, we scale the entries of that input with a scale-vector and we shift the location of the normalized, scaled, centered mini-batch with vector addition. We learn the entries of the shift vector and we learn the entries of the scale-vector. This forcefully ensures that the layers are receiving normalized inputs, and so on the backward sweep there is not a vanishing or exploding gradient that builds up or falls apart, respectively, that is, you can think of an exploding gradient as a snowball rolling down a snowy hill, where the snow is the large partial derivates which multiply to cause an exploding gradient on the backward sweep. Vanishing gradients are the same phenomenon but the snowball gets smaller, and so can perhaps be described as rolling through a blazing hot desert, and so the smaller the ball gets, the easier it melts and multiplicatively so.

Batch normalization has more of an effect the deeper the network. Here is how we can implement batch normalization (BN):
Notice that we simply place the BN before each activation function, even for the output layer.

In [4]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3

In [5]:
# How to look at the names of the variables in the layer
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

Batch normalization is so popular, as of 2019, that neural nets are so assumed to have BN layers before every activation layer that ANN diagrams don't even include the BN, since it is just assumed that they will be included. So, BNs are typically the way to go.

#### Gradient Clipping
Gradient clipping is the final means, mentioned so far, for solving the problem of exploding or vanishing gradients. In RNNs, implementing BN is typically hard or bad. So, instead, we resort to gradient clipping, which simply determines a magnitude limit for the partial derivatives that go into the gradient vector. E.g., we could set the clipvalue to 1, which would ensure that the smallest and largest partial derivatives could be -1 and 1, respectively. Alternatively, we can use clipnorm, which normalizes the gradient vector if the gradients length (l2 norm) is larger than the value you set clipnorm to.

Implementing gradient clipping is extremely simple with keras:

In [6]:
model.compile(loss = 'mse', optimizer=keras.optimizers.SGD(clipnorm=2.2)) # or some other value, ie, 2.2 is arbitrary
# or
model.compile(loss = 'mse', optimizer=keras.optimizers.SGD(clipvalue=1.3))

# Reusing Pretrained Layers:
If we are training a really deep neural network, then we want to avoid the cost of training as much as possible. So, since really deep nets are typically used for tasks wherein the data has many features and those features are hierarchically organized, we can transfer the lower layers from a neural net that was trained on a similar task. Then, we add more layers and learn the weights for the new layers, which will learn the higher level features of our particular task. For example, if we wanted a neural network for recognizing cars in images of many types, and we had access to a neural network that was categorizing plants, people, vehicles and hundreds of other categories in images, then we could transfer the lower layers in this latter network to our new network. Further, we'd freeze the weights of the layers that have been transfered to the new model so that we only learn weights for the new layers.

We can, in order to test if performance improves, unfreeze the weights on the higher transferred layers and proceed to train them. *If we do that, it is best to use a lower learning rate.* We can treat the freeze status of these weights as a hyperparameter for training, basically. Also, if the inputs for our new task are images of a different size than the images that the older network was trained on, then we'd profit from resizing the images to be the same size as the images on which the first model was trained. Furthermore, if we have a ton of data and time to do computations, then we can transfer less or even no layers. So, the number of layers we transfer is inversely proportional to how much data we have and time we want to spend, but mainly how data we have.

### Transfer Learning with Keras:
Here is how we actually implement transfer learning. We assume model_A is a model that has already been trained on a similar task, and model_B is new.

In [8]:
model_A_old = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model_A_old.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=keras.optimizers.SGD(lr=1e-3))
model_A_old.save('model_A.h5')

In [11]:
model_A = keras.models.load_model('model_A.h5')
model_B_on_A = keras.Sequential(model_A.layers[:-1])   # Taking every layer except the last
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))  # For binary classification

# Notice that model A and model B now share layers. This is like when we set a list
# equal to another list. When we change one list, we change the other. Thus, to avoid this,
# we actually need to make a clone of the layers of model A and a clone of the weights,
# since cloning the layers does not clone the weights. Thus, our revised code is:

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))

In [14]:
# We now want to freeze the weights on model B's transfered layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=keras.optimizers.SGD())

In [ ]:
# We only want to keep the layers frozen for the first few epochs of training,
# just so the new layer does not have totally random connections.
history = model_B_on_A.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=4)

for layer in model_B_on_A.layers[:-1]:
    layer.trainaible = True

# We also want to set the learning rate to be really low, since this avoids
# damaging the reused weights. The default learning rate is 1e-2; we choose 1e-4.

model_B_on_A.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=keras.optimizers.SGD(lr=1e-4))

# Then we do more training, and so on.

# Faster Optimizers (i.e., beyond SGD):
When our network is deep, we want to have some means of traversing paramater space that move faster than SGD.

#### Momentum Optimization:
this is a method that is similar to SGD, except when descend in the fitness landscape, we keep a memory of how fast we were moving on prior moves, so that if we were consistently experiencing large gradients, then we move faster. A good analogy is a bowling ball rolling down a steep hill, whereas SGD is a freakish ball that does not remember how fast it was moving while inhabiting previous states in its trajectory. Thus, momentum optimization can move much faster than SGD. Here is the momentum optimization algorithm: Where m is momentum and grad is the gradient of the cost function wrt the network parameters,

                                   1. m = mb - grad
                                   2. theta = theta - m.
b is how much friction we have on the surface that we are rolling down. So, with large b, we move slower. Typically, b = 0.9, but 0 <= b <= 1.

In [17]:
# implementing momentum optimization is extremely easy with keras:
optimizer = keras.optimizers.SGD(momentum = 0.9)

#### Nesterov Accelerated Gradient (NAG):
This is a slight tweak of momentum optimization that greatly speeds up training. The tweak is as follows. Rather than compute the gradient at the points specified by the parameters, compute the gradient at the points specified by the paramaters plus bm, and so we move in the direction suggested by the momentum vector to compute the gradient. In effect, this means we are adding a bias term to each parameter in the network, wherein that bias is the appropriate entry of the momentum vector scaled by b. That is all that's different.

In [18]:
# Implementing NAG:
optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov=True)

#### AdaGrad (Adaptive Gradient Descent):
This is an inefficient method for actually training large neural nets, but it gives some good conceptual underpinning for understanding good methods. We have a vector S that is similar to a momentum vector, except the way S grows is not like m. Rather, S = S + gradient (x) gradient, where (x) means element-wise multiplication. So, each entry of grad (x) grad is just the square of each entry of grad. Then, theta = theta - eta * (grad (/) sqrt(S + epsilon)), where (/) is element-wise division. So, S grows in such a way as to make the larger partial derivatives have less control over the trajectory of the parameter vector. This works as an adpative learning rate, hence the name. However, the reason AdaGrad is not good for larger neural nets is that it can cause the learning rate to effectively go to zero before any worthy optima has been found. Basically, the effect of AdaGrad is to kill the larger partial derivatives, especially if a particular index of the gradient vector has had a history of having large entries (i.e., that dimension of the parameter space is very steep), since then that entry of S gets really big.

#### RMSProp:
This works like AdaGrad except S exponentiall decays, and so we forget about the older history of S. Therefore, steep slopes in parameter space are not punished as severely. Therefore, RMSProp does not kill the learning rate and so is suitable for usage on large neural networks, and in fact was the most widely used method until Adam and Nadam optimization were discovered. Anyways, the term for S is now:

                               S =  rho * S + (1 - rho) * (gradient (x) gradient)
                               theta =  theta - eta * (gradient (/) sqrt(S + epsilon))

In [21]:
RMSPoptimizer = keras.optimizers.RMSprop(rho = 0.9)

#### Adam (Adaptive Moment Estimation):
This is the most popular optimizer. Adam is a mixture of RMSProp and momentum optimization. So, the learning rate is adaptive, does not kill the learning rate since we forget the history of S (unlike AdaGrad), and the momentum increases our speed of descent, causing training to go faster. So, we have b1 and b2 as our parameters, which range from 0 to 1:

                    1. m = m * b1 + (1 - b1) * gradient
                    2. S = S * b2 + (1 - b2) * (gradient (x) gradient)
                    3. m^ = m / (1 - b1^t)      4. S^ = S / (1 - b2^t)    (t is the iteration number, starting at 1)
                    5. theta = theta - (eta * m^) (/) sqrt(S^ + epsilon))

b1 = 0.9 and b2 = 0.999 and eta = 0.001 by default. Epsilon is something like 1e-7. Steps 3 and 4 are simply to boost m and S at the beginning of training, since they are initialized to zero.

In [25]:
adam_optimizer = keras.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)  # These are the default values

Nadam is Adam with the Nesterov trick, that is, the gradient is computed on the parameters plus b1 * m.

### Summary:
So, for deep neural networks, you should treat the choice of optimizer as a hyperparameter. Though, you're most likely best to default to Adam or RMSProp. Some parameter spaces are just better for adam and some are better for RMSProp and some are better for some other optimizer. There does not exist a theory for why, afaik, but I assume it has to do with the topology of the fitness landscape of certain tasks and the ways in which these optimizers cause the model to traverse that landscape.

## Learning Rate Scheduling:
We can change our learning rate according to a schedule. For example, we could start high then get smaller slowly. Or we could do something like eta = - (100 - t)^2 where t is our iteration number. So, there are a lot of different ways we could change the learning rate throughout the training. Ultimately, our goal is to select a learning rate that allows the model to traverse the parameter space in such a way that plateaus and local minima are danced over (large learning rate then) and the global minimum or very good local minima are where we terminate. So, we can imagine that there may exist some fitness landscape topologies where erratic learning rate schedules would actually be optimal--but figuring out when this is the case is likely extremely difficult, and I note it moreso for intellectual interest.

####  Power Scheduling:
This is when eta is a function of the iteration number t. Let eta be n. So, n(t) = n0 / (1 + t/s)^c where n0 is our initial choice of eta, and s and c are hyperparameters.

#### Exponential Scheduling:
We have n(t) = n0 * 0.1^(t/s), and so n(t) is shrunk to 1/10th its size every s iterations.

#### Piecewise Scheduling:
We manually adjust the learning rate every a, b, c, d, ... steps. So, for a steps, we have na, then for b steps we have nb, then for c steps we have nc, etc., where na, nb, nc are all constants. This function is like a staircase where the stairs can be of arbitrary heights (nb) wrt eachother and can be of arbitrary length (b).

#### Performance Scheduling:
This is sort of similar to early stopping. We measure the validation error, and once the validation error has not improved at all for however many steps we specify (like the patience hyperparameter in the EarlyStopping callback), we drop eta by some amount.

#### 1Cycle Scheduling:

We have two hyperparameters, n0 and n1. 1Cycle thus works by starting with n = n0 and then linearly increasing up to n1 by halfway through the training session. Then, at halfway through training, the learning rate n starts to linearly decay back to n0. So, we can visualize this like a linear tent /\\.

In [48]:
# Power scheduling is the simplest to implement:
power_scheduling_optimizer = keras.optimizers.SGD(decay=1e-4)

# Other learning rate scheduling is implemented by making a custom learning rate scheduler callback.
def exp_decay(n0, s):
    def exp_decay_fn(epoch):
        return n0 * 0.1**(epoch / s)
    return exp_decay_fn

power_lr_schedule = keras.callbacks.LearningRateScheduler(exp_decay(0.01, 20))
    # NOTICE: The epoch argument will default to zero everytime we start training, and so you'll have to
    # manually set the epoch to the value you desire when you begin training again. This is done with the
    # fit() methods initial_epoch argument, i.e., set initial_epoch to the value you desire when training again.

# The piecewise schedule:
def piecewise_schedule(epoch):        
    if epoch < 5:
        return 0.1
    elif epoch < 15:
        return 0.01
    return 0.001

piecewise_lr_schedule = keras.callbacks.LearningRateScheduler(piecewise_schedule)

# Performance scheduling:
performance_lr_schedule = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    # This causes the learning rate to be multiplied by factor if there is no improvement in best validation loss for 5 epochs.
    # I think it would be a good idea to use the early scheduling callback in combination with this but to then set
    # the patience hyperparameter of the EarlyStopping() callback to be greater than the patience of the performance schedule.

# Avoiding Overfitting Through Regularization:

### Dropout:
The idea of dropout is simple. We have some probability p called the **dropout rate** for all neurons, usually somewhere between 10 and 50 percent (though, for RNNs, it's between 20 to 30 %, for CNNs it's 40 to 50%). And thus each neuron has a probability p of being ignored during a *training step*, such that its weights are not updated during that training step. (Recall that a *training step* is updating theta based on the partial derivatives obtained from a single batch, whereas an *epoch* consists of however many training steps are needed to go through the entire dataset.) That's it. However, since the neurons, during training, will on average receive (1 - p) * (number of neurons in preceding layer), beyond training, we must divide the signal each neuron receives by (1 - p), so that the neuron is not receiving a signal that is 1/(1 - p) times larger than what it was trained on. Keras handles this automatically.

###### Intuition for dropout:
Forcing neurons to dropout means that neurons cannot come to depend on any single neuron in the network, and so each neuron is forced to be more resilient. This is an example of antifragility. It'd be like if, in a company, employees flipped a biased coin to determine if they should or should not come to work. This strategy would mean that workers could not come to rely on any particular coworkers but would have to learn how to operate with anyone and rely on their selves (i.e., the neurons have to form more or less equal reliance on connected neurons for inputs rather than coming to focus on only one). Another intution for why this may work is that, by doing dropout, we are effectively training multiple, though overlapping, neural networks. Thus, we take the ensemble vote from the networks, analogous to ensemble learning methods.

In [51]:
# Implementing dropout is as simple as adding extra layers. However, in practice
# it is best to use dropout on the top one to three layers.
# Also, if we are using normalization methods, such as selu or batch norm, then we need to use alpha dropout, which is
# a way of doing dropout that retains the normalization of the signal.

dropout_model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dropout(rate=0.2),                  # Notice that we can do dropout *before* the output layer
    keras.layers.Dense(10, activation='softmax')
])

# General Guidelines for Making Deep Neural Nets:
Remember to normalize your inputs!

In [18]:
pd.DataFrame({'Hyperparameter': pd.Series(['kernel initializer', 'Activation function', 'normalization', 'Regularization', 'Optimizer', 'learning rate schedule'], index=range(0, 6)),'Default Value Suggestion': pd.Series(['He initialization', 'ELU', 'None if shallow; Batch Norm if deep', 'Early stopping', 'RMSProp or Nadam or Momentum optimization',  '1cycle'])})

,Hyperparameter,Default Value Suggestion
0,kernel initializer,He initialization
1,Activation function,ELU
2,normalization,None if shallow; Batch Norm if deep
3,Regularization,Early stopping
4,Optimizer,RMSProp or Nadam or Momentum optimization
5,learning rate schedule,1cycle
